In [1]:
# Logging setup
import logging

logging.basicConfig(level=logging.INFO)
logging.getLogger().setLevel(level=logging.ERROR)
logging.getLogger('orion').setLevel(level=logging.INFO)

import warnings
warnings.simplefilter("ignore")

import os

## Creating an instance of the OrionDBExplorer

In order to connect to the database, all you need to do is import and create an instance of the
`OrionDBExplorer` class.

Note that, because of the dynamic schema-less nature of MongoDB, no database initialization
or table creation is needed. All you need to do start using a new database is create the
`OrionDBExplorer` instance with the right connection details and start using it!

In order to create the `OrionDBExplorer` instance you will need to pass:

* `user`: An identifier of the user that is running Orion.
* `database`: The name of the MongoDB database to use. This is optional and defaults to `orion`.

In [2]:
from orion.db import OrionDBExplorer

orex = OrionDBExplorer(user='dyu', database='orion-vis')

This will directly create a connection to the database named `'orion'` at the default
MongoDB host, `localhost`, and port, `27017`.

In case you wanted to connect to a different database, host or port, or in case user authentication
is enabled in your MongoDB instance, you can pass a dictionary or a path to a JSON file containing
any required additional arguments:

* `host`: Hostname or IP address of the MongoDB Instance. Defaults to `'localhost'`.
* `port`: Port to which MongoDB is listening. Defaults to `27017`.
* `username`: username to authenticate with.
* `password`: password to authenticate with.
* `authentication_source`: database to authenticate against.

Once we have created the `OrionDBExplorer` instance, and to be sure that we are ready to follow
the tutorial, let's do the following two set-up setps:

1. Drop the `orion-usage-example` database

**WARNING**: This will remove all the data that exists in this database!

In [3]:
orex.drop_database()

2. Make sure to have downloaded some demo data using the `orion.data.download_demo()` function

In [4]:
from orion.data import download_demo

download_demo()

INFO:orion.data:Downloading Orion Demo Data to folder orion-data


This will create a folder called `orion-data` in your current directory with the 3 CSV files
that we will use later on.

## Setting up the Orion Environment

The first thing that you will need to do to start using **Orion** with a Database will be
to add information about your data and your pipelines.

This can be done by using the methods of the `OrionDBExplorer` class that are documenteted below,
which allow creating the corresponding objects in the Database.

### Add a Dataset

In order to add a dataset you can use the `add_dataset` method, which has the following arguments:

* `name (str)`: Name of the dataset
* `entity (str)`: Name or Id of the entity which this dataset is associated to

Let's create the `Demo Dataset` that we will use for our demo.

In [5]:
dataset = orex.add_dataset(
    name='nasa demo',
    entity='Orion',
)

This call will try to create a new _Dataset_ object in the database and return it.

We can now see the _Dataset_ that we just created using the `get_datasets` method:

In [6]:
orex.get_datasets()

,dataset_id,created_by,entity,insert_time,name
0,61842df84c3fefe0d137ebde,dyu,Orion,2021-11-04 19:01:11.864,nasa demo


### Add Signals

Additionally, we can also add all the signals that exist inside a folder by using the `add_signals`
method, passing a `signals_path`:

In [7]:
orex.add_signals(
    dataset=dataset,
    signals_path='orion-data'
)

After this is done, we can see that one signal has been created for each one of the CSV
files that we downloaded before.

In [9]:
orex.get_signals(dataset=dataset)

,signal_id,created_by,data_location,dataset,insert_time,name,start_time,stop_time
0,618421ce99b30d280a424110,dyu,orion-data/S-1.csv,6184219b99b30d280a42410f,2021-11-04 18:09:18.243,S-1,1222819200,1442016000
1,618421de99b30d280a424112,dyu,orion-data/P-1.csv,6184219b99b30d280a42410f,2021-11-04 18:09:34.965,P-1,1222819200,1468540800
2,618421de99b30d280a424113,dyu,orion-data/E-1.csv,6184219b99b30d280a42410f,2021-11-04 18:09:34.978,E-1,1222819200,1468951200


### Add a Template

The next thing we need to add is a _Template_ to the Database using the `add_template` method.

This method expects:

* `name (str)`: Name of the template.
* `template (dict or str)`: Optional. Specification of the template to use, which can be one of:
    * An MLPipeline instance
    * The name of a registered template
    * a dict containing the MLPipeline details
    * The path to a pipeline JSON file.
    
**Orion** comes with a few templates ready to be used, so let's have a look at the ones that exist
using the `orion.get_available_templates` function.

In [8]:
from orion.analysis import get_available_templates

get_available_templates()

['azure',
 'arima',
 'dense_autoencoder',
 'tadgan',
 'lstm_autoencoder',
 'lstm_dynamic_threshold',
 'dummy']

And now let's create a _Template_ using the `lstm_dynamic_threshold` template.

In [9]:
import os
file = os.path.abspath('../orion/pipelines/verified/lstm_dynamic_threshold/lstm_dynamic_threshold_vis.json')

print(file)

/Users/dyu/Projects/dai_sintel/orion/orion/pipelines/verified/lstm_dynamic_threshold/lstm_dynamic_threshold_vis.json


In [10]:
template = orex.add_template(
    name='lstmdt',
    template=file,
)

Using TensorFlow backend.


We can now see the _Template_ that we just created

Also, during this step, apart from a _Template_ object, a _Pipeline_ object has also been
registred with the same name as the _Template_ and using the default hyperparameter values.

However, if we want to use a configuration different from the default, we might want to
create another _Pipeline_ with custom hyperparameter values.

In order to do this we will need to call the `add_pipeline` method passing:

* `name (str)`: Name given to this pipeline
* `template (Template or ObjectID)`: Template or the corresponding id.
* `hyperparameters (dict or str)`: dict containing the hyperparameter details or path to the
  corresponding JSON file. Optional.

For example, if we want to specify a different number of epochs for the LSTM primitive of the
pipeline that we just created we will run:

In [12]:
new_hyperparameters = {
   'keras.Sequential.LSTMTimeSeriesRegressor#1': {
       'epochs': 1,
       'verbose': True
   }
}
pipeline = orex.add_pipeline(
   name='lstmdt_1_epoch',
   template=template,
   hyperparameters=new_hyperparameters,
)

And we can see how a new _Pipeline_ was created in the Database.

In [13]:
orex.get_pipelines()

,pipeline_id,created_by,insert_time,name,template
0,61842e444c3fefe0d137ebe3,dyu,2021-11-04 19:02:28.677,lstmdt,61842e444c3fefe0d137ebe2
1,61842e524c3fefe0d137ebe4,dyu,2021-11-04 19:02:42.599,lstmdt_1_epoch,61842e444c3fefe0d137ebe2


### Add an Experiment

Once we have a _Dataset_ with _Signals_ and a _Template_, we are ready to add an
_Experiment_.

In order to run an _Experiment_ we will need to:

1. Get the _Dataset_ and the list of _Signals_ that we want to run the _Experiment_ on.
2. Get the _Template_ which we want to use for the _Experiment_
3. Call the `add_experiment` method passing all these with an experiment, a project name and a
   username.

For example, if we want to create an experiment using the _Dataset_, the _Signals_ and the
_Template_ that we just created, we will use:

In [14]:
experiment = orex.add_experiment(
    name='My Experiment',
    project='My Project',
    template=template,
    dataset=dataset,
)

This will create an _Experiment_ object in the database using the indicated _Template_
and all the _Signals_ from the given _Dataset_.

In [15]:
orex.get_experiments()

,experiment_id,created_by,dataset,insert_time,name,project,signals,template
0,61842e5a4c3fefe0d137ebe5,dyu,61842df84c3fefe0d137ebde,2021-11-04 19:02:50.294,My Experiment,My Project,"[61842e0d4c3fefe0d137ebdf, 61842e0d4c3fefe0d13...",61842e444c3fefe0d137ebe2


## Starting a Datarun

Once we have created our _Experiment_ object we are ready to start executing _Pipelines_ on our
_Signals_.

For this we will need to use the `orion.runner.start_datarun` function, which expects:

* `orex (OrionExplorer)`: The `OrionDBExplorer` instance.
* `experiment (Experiment or ObjectID)`: Experiment object or the corresponding ID.
* `pipeline (Pipeline or ObjectID)`: Pipeline object or the corresponding ID.

This will create a _Datarun_ object for this _Experiment_ and _Pipeline_ in the database,
and then it will start creating and executing _Signalruns_, one for each _Signal_ in the _Experiment_.

Let's trigger a _Datarun_ using the `lstmdt_1_epoch` _Pipeline_ that we created.

In [16]:
from orion.runner import start_datarun

start_datarun(orex, experiment, pipeline)

INFO:orion.runner:Datarun 61842e634c3fefe0d137ebe6 started
INFO:orion.runner:Signalrun 61842e644c3fefe0d137ebe7 started
INFO:orion.runner:Running pipeline lstmdt_1_epoch on signal S-1


a
Train on 7919 samples, validate on 1980 samples
Epoch 1/1
9899/9899 [==============================] - 11s 1ms/step


INFO:orion.runner:Processing pipeline lstmdt_1_epoch predictions on signal S-1
INFO:orion.runner:Signalrun 61842e924c3fefe0d137ebf7 started
INFO:orion.runner:Running pipeline lstmdt_1_epoch on signal P-1


a
Train on 8901 samples, validate on 2226 samples
Epoch 1/1
11127/11127 [==============================] - 16s 1ms/step


INFO:orion.runner:Processing pipeline lstmdt_1_epoch predictions on signal P-1
INFO:orion.runner:Signalrun 61842f0a4c3fefe0d137ec06 started
INFO:orion.runner:Running pipeline lstmdt_1_epoch on signal E-1


a
Train on 8916 samples, validate on 2230 samples
Epoch 1/1
11146/11146 [==============================] - 30s 3ms/step


INFO:orion.runner:Processing pipeline lstmdt_1_epoch predictions on signal E-1


## Launch Sintel — the python server providing RESTful APIs  

https://github.com/sintel-dev/sintel


- step1: install
- step2: go data section and upgrade ORION db to sintel db
- step3: run server

## Launch MTV

https://github.com/sintel-dev/MTV

- step1: go client folder and run `npm install`
- step2: 